## Data Source

https://api.covid19india.org/  
https://api.covid19india.org/csv/

Total Confirmed Cases - https://public.flourish.studio/visualisation/2917845/ <br>
Total Recovered - https://app.flourish.studio/visualisation/2917909/ <br>
Confirmed vs Recovered - https://app.flourish.studio/visualisation/2917932/ <br>
Confirmed vs Deceased - https://app.flourish.studio/visualisation/2917972/ <br>

## Import libraries

In [14]:
import re
import requests
import json
import math
import csv
import numpy as np
import pandas as pd
import folium
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sys, getopt, pprint
from pymongo import MongoClient

## Daily cases

In [15]:
day_wise = pd.read_csv('https://api.covid19india.org/csv/latest/case_time_series.csv')

day_wise.to_csv('./data/nation_level_daily.csv', index=False)

day_wise.tail()

,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
152,30 June,18256,585795,12565,347840,506,17411
153,01 July,19429,605224,12064,359904,438,17849
154,02 July,21947,627171,19999,379903,378,18227
155,03 July,22718,649889,14417,394320,444,18671
156,04 July,24018,673907,14743,409063,610,19281


In [16]:
def timeformat(x):
    d = datetime.strptime(x, '%d %B , %Y')
    x = d.strftime('%Y-%m-%d')
    return x

In [17]:
day_wise["Date"] = day_wise["Date"].apply(lambda x: x+", 2020")

In [18]:
day_wise["Date"] = day_wise["Date"].apply(timeformat)

In [19]:
day_wise

,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
0,2020-01-30,1,1,0,0,0,0
1,2020-01-31,0,1,0,0,0,0
2,2020-02-01,0,1,0,0,0,0
3,2020-02-02,1,2,0,0,0,0
4,2020-02-03,1,3,0,0,0,0
...,...,...,...,...,...,...,...
152,2020-06-30,18256,585795,12565,347840,506,17411
153,2020-07-01,19429,605224,12064,359904,438,17849
154,2020-07-02,21947,627171,19999,379903,378,18227
155,2020-07-03,22718,649889,14417,394320,444,18671


In [35]:
#CSV to JSON Conversion
csvfile = open('./data/nation_level_daily.csv', 'r')
reader = csv.DictReader( csvfile )
mongo_client=MongoClient() 
db=mongo_client.national_level_daily
db.segment.drop()

header = day_wise.columns
for each in reader:
    row={}
    for field in header:
        row[field]=each[field]

    db.segment.insert(row)

/home/adi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [20]:
no_rows = len(day_wise)-1
perspective_deaths = math.floor(day_wise["Daily Deceased"][no_rows]/day_wise["Daily Confirmed"][no_rows]*100)

In [21]:
print(str(perspective_deaths)+" in 100 people die due to COVID-19 in India based on the current stats.")

'2 in 100 people die due to COVID-19 in India based on the current stats.'

## Latest - State wise 

In [22]:
latest = pd.read_csv('https://api.covid19india.org/csv/latest/state_wise.csv')

latest.to_csv('./data/state_level_latest.csv', index=False)
latest.drop(columns=["Migrated_Other","Delta_Confirmed","Delta_Recovered","Delta_Deaths","State_Notes"],axis=1,inplace=True)
latest.drop(latest.index[0],inplace=True)
latest.drop(latest.index[9],inplace=True)
latest.reset_index(drop=True)

,State,Confirmed,Recovered,Deaths,Active,Last_Updated_Time,State_code
0,Maharashtra,200064,108082,8671,83295,04/07/2020 21:44:30,MH
1,Tamil Nadu,107001,60592,1450,44959,04/07/2020 19:13:26,TN
2,Delhi,97200,68256,3004,25940,04/07/2020 19:49:26,DL
3,Gujarat,35398,25414,1926,8058,04/07/2020 21:12:26,GJ
4,Uttar Pradesh,26554,18154,773,7627,04/07/2020 21:44:32,UP
5,Rajasthan,19532,15640,447,3445,04/07/2020 21:37:31,RJ
6,West Bengal,21231,14166,736,6329,04/07/2020 20:15:27,WB
7,Madhya Pradesh,14604,11234,598,2772,04/07/2020 19:49:27,MP
8,Haryana,16548,12257,260,4031,04/07/2020 21:03:29,HR
9,Karnataka,21549,9246,335,11964,04/07/2020 20:25:26,KA


In [23]:
df = pd.read_html("https://distancelatlong.com/country/india#table table-striped setBorder")[2]

In [24]:
df

,States,Latitude,Longitude
0,Andaman And Nicobar (1),11.667026,92.735983
1,Andhra Pradesh (20),14.750429,78.570026
2,Arunachal Pradesh (1),27.100399,93.616601
3,Assam (6),26.749981,94.216667
4,Bihar (6),25.785414,87.479973
5,Chandigarh (1),30.719997,76.780006
6,Chhattisgarh (3),22.090420,82.159987
7,Dadra And Nagar Haveli (12),20.266578,73.016618
8,Delhi (2),28.669993,77.230004
9,Goa (1),15.491997,73.818001


In [25]:
latest = latest.sort_values('State')

In [26]:
line = pd.DataFrame({"States": "Ladakh", "Latitude": 34.209515, "Longitude":77.615112}, index=[17])
df2 = pd.concat([df.iloc[:16], line, df.iloc[16:]]).reset_index(drop=True)
line1 = pd.DataFrame({"States": "Gujarat", "Latitude": 22.2587, "Longitude":71.1924}, index=[11])
df3 = pd.concat([df2.iloc[:10], line1, df2.iloc[10:]]).reset_index(drop=True)
line2 = pd.DataFrame({"States": "Telangana", "Latitude": 18.1124, "Longitude":79.0193}, index=[32])
df4 = pd.concat([df3.iloc[:31], line2, df3.iloc[31:]]).reset_index(drop=True)
df4.sort_values("States")
df4.reset_index(drop=True)
df = df4

In [27]:
df = df.sort_values("States")
df.drop(columns="States",axis=1,inplace=True)

In [28]:
latest = latest.reset_index(drop=True)
latest = pd.concat([latest,df],axis=1)

In [29]:
latest

,State,Confirmed,Recovered,Deaths,Active,Last_Updated_Time,State_code,Latitude,Longitude
0,Andaman and Nicobar Islands,119,66,0,53,04/07/2020 23:37:32,AN,11.667026,92.735983
1,Andhra Pradesh,17699,8008,218,9473,04/07/2020 14:46:27,AP,14.750429,78.570026
2,Arunachal Pradesh,259,76,1,182,05/07/2020 00:34:28,AR,27.100399,93.616601
3,Assam,11002,6744,14,4241,05/07/2020 01:36:27,AS,26.749981,94.216667
4,Bihar,11457,8488,88,2881,04/07/2020 17:35:28,BR,25.785414,87.479973
5,Chandigarh,459,395,6,58,04/07/2020 17:58:27,CH,30.719997,76.780006
6,Chhattisgarh,3161,2526,14,621,04/07/2020 23:37:27,CT,22.090420,82.159987
7,Dadra and Nagar Haveli and Daman and Diu,341,142,0,198,04/07/2020 17:50:30,DN,20.266578,73.016618
8,Delhi,97200,68256,3004,25940,04/07/2020 19:49:26,DL,28.669993,77.230004
9,Goa,1684,825,6,853,04/07/2020 20:56:41,GA,15.491997,73.818001


In [30]:
latest["Last_Updated_Time"] = latest["Last_Updated_Time"].apply(lambda x:x.split()[0])

In [31]:
latest = latest.astype(str)
latest.to_csv("latest.csv",index=False)

In [38]:
#CSV to JSON Conversion
csvfile = open('latest.csv', 'r')
reader = csv.DictReader( csvfile )
mongo_client=MongoClient() 
db=mongo_client.latest
db.segment.drop()

header = latest.columns
for each in reader:
    row={}
    for field in header:
        row[field]=each[field]

    db.segment.insert_one(row)

In [32]:
india = folium.Map(location = [20.5937,78.9629],zoom_start=4.5)

for date,state,lat,long,total_cases,Death,Recov,Active in zip(list(latest["Last_Updated_Time"]),list(latest['State']),list(latest['Latitude']),list(latest['Longitude']),list(latest['Confirmed']),list(latest['Deaths']),list(latest['Recovered']),list(latest['Active'])):
    folium.CircleMarker(location = [lat,long],
                       radius = 5,
                       color='red',
                       fill = True,
                       fill_color="red").add_to(india)
    folium.Marker(location = [lat,long],
                  popup=folium.Popup(('<strong><font color= blue>Last Updated: '+date+'</font></striong><br>' +
                    '<strong><b>State  : '+state+'</strong> <br>' +
                    '<strong><b>Total Cases : '+total_cases+'</striong><br>' +
                    '<strong><font color= red>Deaths : </font>'+Death+'</striong><br>' +
                    '<strong><font color=green>Recoveries : </font>'+Recov+'</striong><br>'+
                    '<strong><b>Active Cases : '+Active+'</striong>'  ),max_width=200)).add_to(india)
india

In [33]:
india.save(outfile="india.html")